In [1]:
#| export
verbose                       = 1
check_memory_usage            = True
time_flag                     = True
window_size_percentage        = False
show_plots                    = True
reset_kernel                  = False
pre_configured_case           = True
case_id                       = 1
frequency_factor              = 15
frequency_factor_change_alias = True
check_parameters              = True
cuda_device                   = 0

In [2]:
#| export
# This is only needed if the notebook is run in VSCode
import sys
import dvats.utils as ut
if '--vscode' in sys.argv:
    print("Executing inside vscode")
    ut.DisplayHandle.update = ut.update_patch

In [3]:
p#| p
import dvats.config as cfg_

In [4]:
#| export
import warnings
warnings.filterwarnings("ignore", module="umap")
import os
import sys
sys.path.append(os.path.abspath('..'))
from dvats.all import *
from fastcore.all import *
from tsai.basics import *
from tsai.models.InceptionTimePlus import *
from tsai.callback.MVP import *
import matplotlib.colors as colors
from fastai.callback.wandb import WandbCallback
from fastai.callback.progress import ShowGraphCallback
from fastai.callback.schedule import *
from fastai.callback.tracker import EarlyStoppingCallback
import wandb


Octave is ready <oct2py.core.Oct2Py object at 0x7f38b5f2e5f0>

















In [5]:
#| export
wandb_api = wandb.Api()

In [6]:
#| export
device = torch.device(f'cuda:{cuda_device}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
if check_memory_usage:
    gpu_device = torch.cuda.current_device()
    print("GPU Device", gpu_device)
    print("Device", device)
    gpu_memory_status(gpu_device)

GPU Device 0
Device cuda:0
GPU | Used mem: 0
GPU | Used mem: 24
GPU | Memory Usage: [--------------------] 0%


In [7]:
target_directory = os.path.expanduser("~/work/nbs_pipeline")
print(target_directory)
%cd {target_directory}

/home/macu/work/nbs_pipeline
/home/macu/work/nbs_pipeline


In [8]:
#| export
user, project, version, data, config, job_type = cfg_.get_artifact_config_MVP(False)
if pre_configured_case: 
    cfg_.force_artifact_config_mvp(
        config = config,
        id = case_id,
        verbose = verbose, 
        both = verbose > 0,
        frequency_factor = frequency_factor,
        frequency_factor_change_alias = frequency_factor_change_alias
    )

data_fpath is missing in original dict | ~/data/solar_4_seconds_dataset.tsf 
analysis_mode: online
norm_use_by_single_batch is missing in original dict | (False,) 
mask_sync: False
wandb_group: None
batch_size: 512
train_artifact: mi-santamaria/deepvats/Synthetic_Segmentation_I:latest -> mi-santamaria/deepvats/solar_4_seconds-1m:latest
mask_stateful: True
stride: 9 -> 1
mvp_ws: (25, 50) -> [450, 900]
w: 50 -> 900
valid_size: 0.2
valid_artifact: None
r: 0.71
artifact_name is missing in original dict | solar_4_seconds-1m 
csv_config is missing in original dict | {} 
use_wandb: True
resampling_freq is missing in original dict | 1T 
data_cols is missing in original dict | [] 
norm_use_single_batch: False
alias: Synthetic_Segmentation_I -> solar_4_seconds
freq is missing in original dict | 4s 
mask_future: False
norm_by_sample: False
time_col is missing in original dict | None 
epochs: 100


In [9]:
#| export
path = os.path.expanduser("~/work/nbs_pipeline/")
name="06_MPlots"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
if verbose > 0: print("runname: "+runname)
if verbose > 0: cfg_.show_attrdict(config)

runname: 06_MPlots
alias: solar_4_seconds
analysis_mode: online
batch_size: 512
epochs: 100
mask_future: False
mask_stateful: True
mask_sync: False
mvp_ws: [450, 900]
norm_by_sample: False
norm_use_single_batch: False
r: 0.71
stride: 1
train_artifact: mi-santamaria/deepvats/solar_4_seconds-1m:latest
valid_artifact: None
use_wandb: True
valid_size: 0.2
w: 900
wandb_group: None
artifact_name: solar_4_seconds-1m
data_cols: []
data_fpath: ~/data/solar_4_seconds_dataset.tsf
freq: 4s
time_col: None
csv_config: {}
resampling_freq: 1T
norm_use_by_single_batch: (False,)


In [10]:
#| export
if verbose > 0: print("--> Wandb init")
run = wandb.init(
    entity           = user,
    # work-nbs is a place to log draft runs
    project          = project,
    group            = config.wandb_group,
    job_type         = job_type,
    allow_val_change = True,
    mode             = config.analysis_mode,
    config           = config,
    # When use_wandb is false the run is not linked to a personal account
    #NOTE: This is not working right now
    anonymous        = 'never' if config.use_wandb else 'must', 
    resume           = False,
    name             = runname
)
if verbose > 0: print("Wandb init -->")
config = run.config  # Object for storing hyperparameters
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

--> Wandb init


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find /home/macu/work/nbs_pipeline/06_MPlots.ipynb.
wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


Wandb init -->


In [11]:
#| export
config = run.config  # Object for storing hyperparameters
if verbose > 0: cfg_.show_attrdict(config)
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact
train_artifact = artifacts_gettr(config.train_artifact)
if verbose > 0: print("---> W&B Train Artifact")

alias: solar_4_seconds
analysis_mode: online
batch_size: 512
epochs: 100
mask_future: False
mask_stateful: True
mask_sync: False
mvp_ws: [450, 900]
norm_by_sample: False
norm_use_single_batch: False
r: 0.71
stride: 1
train_artifact: mi-santamaria/deepvats/solar_4_seconds-1m:latest
valid_artifact: None
use_wandb: True
valid_size: 0.2
w: 900
wandb_group: None
artifact_name: solar_4_seconds-1m
data_cols: []
data_fpath: ~/data/solar_4_seconds_dataset.tsf
freq: 4s
time_col: None
csv_config: {}
resampling_freq: 1T
norm_use_by_single_batch: [False]
---> W&B Train Artifact


In [12]:
#| export
df_train = train_artifact.to_df()

wandb:   1 of 1 files downloaded.  


In [13]:
#| export
if verbose > 0: 
    print(df_train.shape)
    display(df_train.head)
    print("df_train ~ ", df_train.shape)
    print("window_sizes = ", config.mvp_ws)
    print("wlen = ", config.w)
    df_train.head

(493149, 1)


<bound method NDFrame.head of                        0
2019-08-01 00:00:03  0.0
2019-08-01 00:01:03  0.0
2019-08-01 00:02:03  0.0
2019-08-01 00:03:03  0.0
2019-08-01 00:04:03  0.0
...                  ...
2020-07-08 11:04:03  0.0
2020-07-08 11:05:03  0.0
2020-07-08 11:06:03  0.0
2020-07-08 11:07:03  0.0
2020-07-08 11:08:03  0.0

[493149 rows x 1 columns]>

df_train ~  (493149, 1)
window_sizes =  [450, 900]
wlen =  900


In [14]:
import dvats.mplots as mplots
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib.gridspec import GridSpec
import pyscamp as scamp
import dvats.memory as mem

In [15]:
mem.gpu_memory_status()

GPU | Used mem: 0
GPU | Used mem: 24
GPU | Memory Usage: [--------------------] 0%


In [16]:
# Allows checking if pyscamp was built with CUDA and has GPU support.
has_gpu_support = scamp.gpu_supported()
has_gpu_support

True

In [17]:
df_train.columns

Int64Index([0], dtype='int64')

In [18]:
# Use step for selecting values in position k*step 
ts_step = 1
ts = df_train.iloc[:, 0].values[::ts_step] #For some reason. Trying to follow MPlot-figures-ref

In [19]:
w = config.w

In [20]:
data_MPlot = mplots.MatrixProfilePlot(
    DM_AB           = mplots.DistanceMatrix(), 
    MP_AB           = mplots.MatrixProfile(),
    data            = ts, 
    data_b          = ts,
    subsequence_len = w,
    self_join       = False
)

In [21]:
data_MPlot.data.shape

(493149,)

In [22]:
from dvats import utils as ut

In [23]:
def patata():
    t = None
    function = ut.funcname()
    t = ut.Time( function = function ) 
    t.start()
    return function
patata()

'patata'

In [24]:
data_MPlot.subsequence_len

900

In [25]:
7397222//900

8219

In [26]:
data_MPlot.compute(
    mp_method           = 'stump_gpu', 
    #mp_method           = "scamp_naive" --> Devuelve siempre 0 en el ejemplo de Pulsus Raxodus | why?
    dm_method           = 'scamp',
    #dm_method           = 'scamp',
    debug               = False,
    time_flag           = True,
    allow_experimental  = True,
    ensure_symetric     = False,
    min_points          = 5000,
    max_points          = 10000,
    subsequence_len     = w,
    provide_len         = False,
    downsample_flag     = True,
    min_lag             = 8,
    verbose             = 3,
    threads             = 1,
    gpus                = [1]
)

[ MatrixProfilePlot | Compute ]
[ MatrixProfilePlot | Compute ] | Distance: z_normalized_euclidean_distance
[ MatrixProfilePlot | Compute ] | Downsample? TA: True TB: True
[ MatrixProfilePlot | Compute ] | points? TA: 5000 - 10000
[ MatrixProfilePlot | Compute ] | points? TB: 5000 - 10000
[ MatrixProfilePlot | Compute ] Timer
[ MatrixProfilePlot | Compute ] | Timer for compute
[ MatrixProfilePlot | Compute ] | Timer Start
[ MatrixProfilePlot | Compute ] | Setup dm method
[ MatrixProfilePlot | Compute ] | Setup mp method
[ MatrixProfilePlot | Compute ] DM method: scamp MP method stump_gpu
MatrixProfilePlot | Compute | No range provided
MatrixProfilePlot | Compute | Range [0:493149, 0:493149]
[ MPlot | Compute ] | -->  Downsample TA
[ MPlot | Compute ] | ---> Downsample TA to 0 : 493149
[ Downsample | Position ] Before | Pos (0, 493149)
[ Downsample | Position ] After | Pos (0, 493149)
[ Downsample | downsample_propose_crop ] Max points: 10000
[ Downsample | downsample_propose_crop ] Min

In [27]:
#Crashes
data_MPlot.compute(
    mp_method           = 'stump_gpu', 
    #mp_method           = "scamp_naive" --> Devuelve siempre 0 en el ejemplo de Pulsus Raxodus | why?
    dm_method           = 'scamp',
    #dm_method           = 'scamp',
    debug               = False,
    time_flag           = True,
    allow_experimental  = False, # Falla con allow experimental
    ensure_symetric     = False,
    subsequence_len     = w, 
    provide_len         = False,
    downsample_flag     = False,
    min_lag             = 8,
    verbose             = 3,
    threads             = 1,
    gpus                = [1],
    compute_mp          = False
)

[ MatrixProfilePlot | Compute ]
[ MatrixProfilePlot | Compute ] | Distance: z_normalized_euclidean_distance
[ MatrixProfilePlot | Compute ] | Downsample? TA: False TB: False
[ MatrixProfilePlot | Compute ] | points? TA: 1 - 10000
[ MatrixProfilePlot | Compute ] | points? TB: 1 - 10000
[ MatrixProfilePlot | Compute ] Timer
[ MatrixProfilePlot | Compute ] | Timer for compute
[ MatrixProfilePlot | Compute ] | Timer Start
[ MatrixProfilePlot | Compute ] | Setup dm method
[ MatrixProfilePlot | Compute ] | Setup mp method
[ MatrixProfilePlot | Compute ] DM method: scamp MP method stump_gpu
MatrixProfilePlot | Compute | No range provided
MatrixProfilePlot | Compute | Range [0:493149, 0:493149]
MPlot | Compute | Do not downsample => use original TA time series
MPlot | Compute | Do not downsample => use original TB time series
MPlot | Compute | --> Ensure parameters 
MPlot | Compute | --> provide_len 
MPlot | Compute | --> provide_len ... 3 ...
[ MPlot | Compute ] | Ensure Parameters TB_paa ~ 4

MemoryError: Unable to allocate 1.76 TiB for an array with shape (492250, 492250) and data type float64

In [ ]:
"""
data_MPlot.compute(
    mp_method           = 'stump_gpu', 
    #mp_method           = "scamp_naive" --> Devuelve siempre 0 en el ejemplo de Pulsus Raxodus | why?
    dm_method           = 'stump',
    #dm_method           = 'scamp',
    debug               = False,
    time_flag           = True,
    allow_experimental  = True,
    ensure_symetric     = False,
    max_points          = 10000,
    subsequence_len     = w, #1 heartbeat
    provide_len         = False,
    downsample_flag     = True,
    min_lag             = 8,
    verbose             = verbose,
    threads             = 1,
    gpus                = [1]
)
"""

In [ ]:
"""
data_MPlot.compute(
    mp_method           = 'stump_gpu', 
    #mp_method           = "scamp_naive" --> Devuelve siempre 0 en el ejemplo de Pulsus Raxodus | why?
    dm_method           = 'stump',
    #dm_method           = 'scamp',
    debug               = False,
    time_flag           = True,
    allow_experimental  = True,
    ensure_symetric     = False,
    max_points          = np.inf,
    subsequence_len     = w, #1 heartbeat
    provide_len         = False,
    downsample_flag     = False,
    min_lag             = 8,
    verbose             = verbose,
    threads             = 1,
    gpus                = [1]
)
"""

In [ ]:
run.finish()